In [45]:
test_input = split("""5483143223
2745854711
5264556173
6141336146
6357385478
4167524645
2176841721
6882881134
4846848554
5283751526""", "\n")

function parse_day11(lines)
    width = length(lines[1])
    height = length(lines)
    cavern = fill(0, (height, width))
    for y in 1:height
        for x in 1:width
            cavern[y,x] = parse(Int8, lines[y][x])
        end
    end
    return cavern
end
test_cavern = parse_day11(test_input)

10×10 Matrix{Int64}:
 5  4  8  3  1  4  3  2  2  3
 2  7  4  5  8  5  4  7  1  1
 5  2  6  4  5  5  6  1  7  3
 6  1  4  1  3  3  6  1  4  6
 6  3  5  7  3  8  5  4  7  8
 4  1  6  7  5  2  4  6  4  5
 2  1  7  6  8  4  1  7  2  1
 6  8  8  2  8  8  1  1  3  4
 4  8  4  6  8  4  8  5  5  4
 5  2  8  3  7  5  1  5  2  6

In [46]:
function neighbors(cavern, position)
    R = CartesianIndices(cavern)
    Ifirst, Ilast = first(R), last(R)
    I1 = oneunit(Ifirst)
    return max(Ifirst, position - I1):min(Ilast, position+I1)
end

function part1(cavern::AbstractArray, steps = 100)
    flashes = 0
    for step in 1:steps
        cavern .+= 1
        flashed = CartesianIndex{2}[]
        new_flashes::AbstractArray{CartesianIndex{2},1} = findall(>(9), cavern)
        while !isempty(new_flashes)
            push!(flashed, new_flashes...)
            for pos in new_flashes
                cavern[neighbors(cavern,pos)] .+= 1
                # Don't increment the flashing location
                cavern[pos] -= 1
            end
            new_flashes = findall(>(9), cavern)
            new_flashes = setdiff(new_flashes, flashed)
        end
        cavern[flashed] .= 0
        # print(flashed)
        flashes += length(Set(flashed))
    end
    return (cavern, flashes)
end


part1 (generic function with 2 methods)

In [48]:
surround_test_cavern = parse_day11(split("""11111
19991
19191
19991
11111""", "\n"))
# part1(surround_test_cavern, 1)

cavern_step1 = parse_day11(split("""6594254334
3856965822
6375667284
7252447257
7468496589
5278635756
3287952832
7993992245
5957959665
6394862637""", "\n"))
# count(==(9), test_cavern)
# part1(test_cavern, 1)
@assert(part1(copy(test_cavern), 1) == (cavern_step1, 0))
cavern_step100 = parse_day11(split("""0397666866
0749766918
0053976933
0004297822
0004229892
0053222877
0532222966
9322228966
7922286866
6789998766""", "\n"))
@assert(part1(copy(test_cavern), 100) == (cavern_step100, 1656))


In [49]:
part1(parse_day11(readlines("inputs/day11.txt")), 100)

([4 4 … 0 0; 4 5 … 0 0; … ; 3 3 … 2 2; 0 3 … 2 8], 1688)

In [71]:
function part2(cavern)
    # So this is funky, Julia passes by value but that means we are passing the matrix pointer by value
    # which is why I needed to `copy` in the above tests so here we can just repeatedly step by 1 
    # without the copy and get the output at each step
    step = 0
    while !all(==(0), cavern)
        part1(cavern, 1)
        step += 1
    end
    return step
end

part2 (generic function with 1 method)

In [72]:
@assert(part2(copy(test_cavern)) == 195)
part2(parse_day11(readlines("inputs/day11.txt")))

403